<a href="https://colab.research.google.com/github/sachincredible9/Python_basics/blob/main/SQL_RAG_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain langchain-community langchain-huggingface streamlit chromadb
!pip install ctransformers[cuda]
!pip install pyngrok
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105

In [3]:
import os
import shutil
# Removed: from huggingface_hub import HfApi, hf_hub_cache_dir # Changed get_hf_cache_home to hf_hub_cache_dir

# Instantiate HfApi (not strictly needed for cache clearing, but kept for context if user adds more HfApi calls)
# api = HfApi()

# Get cache directory path directly
cache_dir = os.path.expanduser('~/.cache/huggingface/hub')

print(f"Hugging Face cache directory: {cache_dir}")

# Remove the cache directory (use with caution, this will delete all downloaded models)
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print("Hugging Face cache cleared successfully.")
else:
    print("Hugging Face cache directory not found.")

Hugging Face cache directory: /root/.cache/huggingface/hub
Hugging Face cache directory not found.


In [4]:
# After running the above cell and ensuring you are logged in,
# you should re-run the cell that starts the Streamlit app to try downloading the model again.
# If the issue persists, there might be a problem with the specific model file or network connectivity.


from huggingface_hub import login
from google.colab import userdata

# Retrieve the Hugging Face token from Colab secrets
hf_token = userdata.get('HF_TOKEN')

if hf_token:
    login(token=hf_token)
    print("Successfully logged in to Hugging Face Hub.")
else:
    print("Hugging Face token (HF_TOKEN) not found in Colab secrets. Please add it.")

Successfully logged in to Hugging Face Hub.


In [7]:
import os
import tempfile
import io
import contextlib

from langchain_community.llms import CTransformers
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

# --- Load Model (Mistral 7B) ---
def load_llm_direct():
    print("Loading Mistral 7B model...")
    llm = CTransformers(
        model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
        model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
        model_type="mistral",
        gpu_layers=50,  # Force GPU usage
        config={
            'max_new_tokens': 1024,
            'context_length': 4096, # Mistral has a larger context window
            'temperature': 0.0
            # Removed 'stop' sequences to allow more verbose output from the LLM
        }
    )
    print("Model loaded successfully.")
    return llm

try:
    llm = load_llm_direct()
except Exception as e:
    print(f"Failed to load model: {e}")
    llm = None

Loading Mistral 7B model...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Model loaded successfully.


In [ ]:
if llm:
    # Assuming 'students.db' is available in the environment
    db_path = './input.db' # Or provide the correct path to your sqlite file
    print(f"Checking for database at: {db_path}")
    if not os.path.exists(db_path):
        print(f"Error: Database file not found at {db_path}")
    else:
        db_uri = f"sqlite:///{db_path}"
        db = SQLDatabase.from_uri(db_uri)
        print(f"Database Connected! Tables: {db.get_usable_table_names()}")

        # Create Agent
        agent_executor = create_sql_agent(
            llm=llm,
            db=db,
            agent_type="zero-shot-react-description",
            verbose=True, # Keep verbose=True to see internal steps
            handle_parsing_errors=True, # Re-enabling to get better error messages
            max_iterations=10 # Increased max_iterations to give more attempts
        )

        print("\nSQL Agent is ready. Type your questions, or type 'exit' to quit.\n")

        while True:
            question = input("Your question: ")
            if question.lower() == 'exit':
                print("Exiting SQL Agent.")
                break

            print(f"\nExecuting agent with question: '{question}'\n")

            # Removed contextlib.redirect_stdout to directly print verbose output
            try:
                response = agent_executor.invoke(question)
                print(f"\n--- Agent Final Response ---\n{response}")
            except Exception as e:
                print(f"\n--- Agent Execution Error ---\nError: {e}")

            # Debug info capture is no longer needed if verbose is printed directly
            print("\n" + "="*50 + "\n") # Separator for clarity

else:
    print("LLM not loaded, skipping agent execution.")

Checking for database at: ./input.db
Database Connected! Tables: ['AllProducts', 'AllProposals', 'AppointmentList', 'AppointmentListReschedule', 'AppointmentNotes', 'AppointmentPhotos', 'AppointmentScannedBarCodes', 'ArriveInspection', 'AuthToken', 'BranchOrStatePricingRecords', 'BranchPostalCodes', 'CheckPoint', 'CommentsForNoticeQuestions', 'CommentsForProposalQuestions', 'CommentsForQuestions', 'Configuration', 'ConversionOptions', 'ConversionTemplateControl', 'CouponCodes', 'CouponParentTemplate', 'CouponParentTemplateItem', 'CouponServiceLineMapping', 'CouponTemplateExclusion', 'CreateLeadAndAppointment', 'CustomerBillingHistrotyInvoices', 'CustomerDetails', 'CustomerProposals', 'CustomerSalesAgreements', 'CustomerServiceHistory', 'DashboardSalesChart', 'DepartReason', 'DepartReasonMaster', 'DeriveStateFindingAndAreaCode', 'EarlyTerminationFeePricing', 'EmailProposal', 'EmployeeLicense', 'EmployeeSkills', 'EquipmentCategory', 'Equipments', 'ErrorLog', 'FileStorageMigrationChecker'